In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict,  Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from imblearn.over_sampling import BorderlineSMOTE
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [5]:
def apply_bsmote_with_limit(X, y, scaling_factor=5000, max_class_limit_ratio=1):
    """
    Apply BorderlineSMOTE with a limit on how much minority classes can be upsampled.

    Args:
        X (pd.DataFrame): Features of the dataset.
        y (pd.Series): Labels of the dataset.
        scaling_factor (int): Maximum upscaling factor for minority classes relative to their original size.
        max_class_limit_ratio (float): Maximum oversampling limit as a fraction of the largest class size.

    Returns:
        tuple: Resampled X and y.
    """

    # Convert y to a pandas Series for easier manipulation
    y = pd.Series(y)
    
    # Dynamically determine n_neighbors based on smallest class
    class_counts = Counter(y)
    min_class_size = min(class_counts.values())
    largest_class_size = max(class_counts.values())
    n_neighbors = min(5, min_class_size - 1)  # Adjust neighbors to fit smallest class
    j_neighbors = min(10, min_class_size - 1)

    # Calculate target sizes for minority classes
    smote_target_sizes = {
        class_label: min(
            scaling_factor * original_size,
            int(largest_class_size * max_class_limit_ratio)
        )
        for class_label, original_size in class_counts.items()
    }

    # Adjust BorderlineSMOTE strategy to limit the oversampling
    smote_strategy = {
        class_label: target_size
        for class_label, target_size in smote_target_sizes.items()
        if target_size > class_counts[class_label]
    }

    # Apply BorderlineSMOTE with the limited strategy
    smote = BorderlineSMOTE(sampling_strategy=smote_strategy, random_state=42, k_neighbors=n_neighbors, m_neighbors=j_neighbors)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    return pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled)


In [6]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle

    # Resample Training Data
    X = train_data.iloc[:, :-1]
    y = train_data.iloc[:, -1]
    del train_data
    gc.collect()
    X_resampled, y_resampled = apply_bsmote_with_limit(X, y)

    # Prepare Training Data
    if (is_string_labels):
        y_resampled = y_resampled.map(label_mapping)
    train_X = X_resampled.to_numpy()
    train_y = y_resampled.to_numpy()
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del X_resampled
    del y_resampled
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 10

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(train_X.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, train_X.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=1000, 
            batch_size=int(train_X.shape[0] * 0.005),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [7]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/10 started...
Run 2/10 started...
Run 3/10 started...
Run 4/10 started...
Run 5/10 started...
Run 6/10 started...
Run 7/10 started...
Run 8/10 started...
Run 9/10 started...
Run 10/10 started...

Aggregated Metrics:
test_loss: 0.7843
test_accuracy: 0.7029
balanced_accuracy: 0.6009
roc_auc: 0.8662
pr_auc: 0.9213
f2: 0.7286
mcc: 0.5228
0_precision: 0.9724
0_recall: 0.7174
0_f1-score: 0.8117
0_support: 9117.0000
1_precision: 0.2429
1_recall: 0.7700
1_f1-score: 0.2736
1_support: 10.0000
2_precision: 0.2306
2_recall: 0.4265
2_f1-score: 0.2616
2_support: 34.0000
3_precision: 0.7107
3_recall: 0.5920
3_f1-score: 0.6161
3_support: 1781.0000
4_precision: 0.8183
4_recall: 0.8173
4_f1-score: 0.8157
4_support: 653.0000
5_precision: 0.2341
5_recall: 0.3500
5_f1-score: 0.2616
5_support: 2.0000
6_precision: 0.1464
6_recall: 0.5333
6_f1-score: 0.1964
6_support: 3.0000
accuracy: 0.7029
macro avg_precision: 0.4794
macro avg_recall: 0.6009
macro avg_f1-score: 0.4624
macro avg_support: 11600.0000
wei

In [8]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/10 started...
Run 2/10 started...
Run 3/10 started...
Run 4/10 started...
Run 5/10 started...
Run 6/10 started...
Run 7/10 started...
Run 8/10 started...
Run 9/10 started...
Run 10/10 started...

Aggregated Metrics:
test_loss: 1.6849
test_accuracy: 0.2800
balanced_accuracy: 0.2844
roc_auc: 0.6364
pr_auc: 0.4341
f2: 0.2521
mcc: 0.0872
0_precision: 0.4012
0_recall: 0.3316
0_f1-score: 0.2805
0_support: 42368.0000
1_precision: 0.5677
1_recall: 0.2695
1_f1-score: 0.2755
1_support: 56661.0000
2_precision: 0.1818
2_recall: 0.0603
2_f1-score: 0.0805
2_support: 7151.0000
3_precision: 0.0871
3_recall: 0.5550
3_f1-score: 0.1271
3_support: 549.0000
4_precision: 0.0159
4_recall: 0.2742
4_f1-score: 0.0293
4_support: 1899.0000
5_precision: 0.0656
5_recall: 0.1500
5_f1-score: 0.0859
5_support: 3473.0000
6_precision: 0.1276
6_recall: 0.3504
6_f1-score: 0.1147
6_support: 4102.0000
accuracy: 0.2800
macro avg_precision: 0.2067
macro avg_recall: 0.2844
macro avg_f1-score: 0.1419
macro avg_support: 11

In [9]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/10 started...
Run 2/10 started...
Run 3/10 started...
Run 4/10 started...
Run 5/10 started...
Run 6/10 started...
Run 7/10 started...
Run 8/10 started...
Run 9/10 started...
Run 10/10 started...

Aggregated Metrics:
test_loss: 0.2258
test_accuracy: 0.9523
balanced_accuracy: 0.0712
roc_auc: 0.5921
pr_auc: 0.9319
f2: 0.9447
mcc: 0.1096
0_precision: 0.9588
0_recall: 0.9940
0_f1-score: 0.9759
0_support: 194557.0000
1_precision: 0.0836
1_recall: 0.1618
1_f1-score: 0.1008
1_support: 3178.0000
2_precision: 0.0800
2_recall: 0.0002
2_f1-score: 0.0003
2_support: 2496.0000
3_precision: 0.2300
3_recall: 0.0005
3_f1-score: 0.0010
3_support: 2083.0000
4_precision: 0.0652
4_recall: 0.0408
4_f1-score: 0.0502
4_support: 463.0000
5_precision: 0.0953
5_recall: 0.0975
5_f1-score: 0.0964
5_support: 441.0000
6_precision: 0.1000
6_recall: 0.0049
6_f1-score: 0.0093
6_support: 204.0000
7_precision: 0.1000
7_recall: 0.0980
7_f1-score: 0.0990
7_support: 196.0000
8_precision: 0.0929
8_recall: 0.0981
8_f1-sc

In [10]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/10 started...
Run 2/10 started...
Run 3/10 started...
Run 4/10 started...
Run 5/10 started...
Run 6/10 started...
Run 7/10 started...
Run 8/10 started...
Run 9/10 started...
Run 10/10 started...

Aggregated Metrics:
test_loss: 1.9090
test_accuracy: 0.3719
balanced_accuracy: 0.2414
roc_auc: 0.6618
pr_auc: 0.7553
f2: 0.3864
mcc: 0.0614
0_precision: 0.8946
0_recall: 0.4106
0_f1-score: 0.4878
0_support: 26862.0000
1_precision: 0.2811
1_recall: 0.1595
1_f1-score: 0.1745
1_support: 2657.0000
2_precision: 0.0571
2_recall: 0.1441
2_f1-score: 0.0688
2_support: 908.0000
3_precision: 0.1419
3_recall: 0.0659
3_f1-score: 0.0358
3_support: 522.0000
4_precision: 0.0153
4_recall: 0.3154
4_f1-score: 0.0233
4_support: 293.0000
5_precision: 0.0431
5_recall: 0.0818
5_f1-score: 0.0318
5_support: 269.0000
6_precision: 0.0100
6_recall: 0.2353
6_f1-score: 0.0138
6_support: 116.0000
7_precision: 0.0439
7_recall: 0.3377
7_f1-score: 0.0747
7_support: 53.0000
8_precision: 0.0284
8_recall: 0.4227
8_f1-score: